In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import yfinance as yf
from machine_learning_finance import analyze_trades, \
    calc_probabilties_without_lookahead, plot_backtest_analysis, \
    metrics_to_dataframe, create_train_test_windows, make_inverse_env_for
from plotly.offline import init_notebook_mode

init_notebook_mode(connected=True)
windows = [300, 600, 900, 1500]

symbol = "XLE"
inverse = "DUG"
file = f"../backtests/backtest_inverse_{symbol}_365_h0.9_l0.1.csv"
period = 365
pd.options.display.max_rows = None


def plot_win_loss(file, metrics):
    ledger = pd.read_csv(file)
    # Set the style for the plots
    sns.set(style='whitegrid')

    # Create a bar plot for profit and loss stats
    profit_stats, loss_stats = metrics['profit_stats'], metrics['loss_stats']
    profit_loss_df = pd.DataFrame([profit_stats, loss_stats], columns=['min', 'max', 'mean', 'median', 'std'], index=['profit', 'loss'])

    plt.figure(figsize=(12, 6))
    ax = sns.barplot(data=profit_loss_df.transpose(), palette='muted')
    ax.set_title('Profit and Loss Statistics')
    ax.set_ylabel('Value')

    # Show the plot
    plt.show()

def analyze_and_graph_range(file, symbol, start, end, df):
    ledger = pd.read_csv(file)
    hist_df, test_df = create_train_test_windows(df, start=start, end=end)    
    test_df = calc_probabilties_without_lookahead(test_df, hist_df, hist_cutoff="2023-05-15")
    plot_backtest_analysis(test_df, ledger, inverse=symbol)
    return ledger, test_df
   
def analyze_and_graph(file, symbol, period, df):
    ledger = pd.read_csv(file)
    hist_df, test_df = create_train_test_windows(df)  
    df = df.tail(period)
    df = calc_probabilties_without_lookahead(df, hist_df)
    plot_backtest_analysis(df, ledger, inverse=symbol)

    return ledger, df

ticker_obj = yf.download(tickers=symbol)
df = pd.DataFrame(ticker_obj)
#plot_win_loss(file, metrics)

#print(ledger)

#print(metrics_df)
#df.sort_index(ascending=False)

[*********************100%***********************]  1 of 1 completed


In [ ]:
ledger, df2 = analyze_and_graph(file, symbol, 365*4, df)    
#ledger, df2 = analyze_and_graph_range(file, symbol, "03-27-2020", "12-17-2021", df)

# Display metrics as text
metrics = analyze_trades(ledger, symbol, period)
metrics["file"] = file
metrics_df = metrics_to_dataframe(metrics) 

merged_df = ledger.merge(df2, left_on='Date', right_index=True, how='left')
merged_df[["Date", "Price", "Product", "Side", "Action","prob_above_trend", "Value"]]

,Date,Price,Product,Side,Action,prob_above_trend,Value
0,2022-01-10,61.761502,XLE,long,enter,0.079274,4938.579999
1,2022-05-17,84.708703,XLE,long,exit,0.123366,6806.334358
2,2022-05-17,18.250700,DUG,long,enter,0.123366,6722.098123
3,2022-07-11,23.199699,DUG,long,exit,0.975533,8608.770659
4,2022-07-11,70.972697,XLE,long,enter,0.975533,8502.924736
5,2022-10-31,90.900000,XLE,long,exit,0.022995,10951.255139
6,2022-10-31,12.291700,DUG,long,enter,0.022995,10815.807400
7,2023-06-23,14.533900,DUG,long,exit,0.660786,12882.685582


In [ ]:
df2 = df2.sort_values('prob_above_trend', ascending=False)
df2

In [4]:
env = make_inverse_env_for(symbol,
                        inverse,
                        1,
                        period,
                        prob_high=0.9,
                        prob_low=0.1)
env.expert_opinion_df()
env.timeseries.tail(35)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,prob_above_trend,trend,trend-diff,action
Date,,,,
2023-05-02,0.743353,0.722639,0.416131,0
2023-05-03,0.829524,0.720762,0.352057,0
2023-05-04,0.870320,0.719997,0.314614,0
2023-05-05,0.762558,0.723537,0.402983,0
2023-05-08,0.767185,0.726674,0.399591,0
2023-05-09,0.766730,0.727626,0.399867,0
2023-05-10,0.816693,0.725709,0.362364,0
2023-05-11,0.858483,0.721016,0.326124,0
2023-05-12,0.847379,0.716002,0.336764,0
